Author: Damir Valput

Date: 4 January 2021

Program: Machine Learning Engineer nanodegree, Udacity

# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

### Machine Learning Workflow

These are the steps that I am going to perform to implement this application:

1. Data loading and exploration
2. Data cleaning and pre-processing
3. Feature engineering and data transformation
4. ML model
5. xx

In [1]:
import pandas as pd
import numpy as np
import math
import json

from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline 

In [82]:
pd.options.mode.chained_assignment = None

In [2]:
# sagemaker libraries
#import sagemaker

# 1. Data loading and exploratory data analysis

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [3]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

#### Portfolio

In [4]:
portfolio.shape

(10, 6)

In [5]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [6]:
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


In [7]:
list(portfolio['offer_type'].unique())

['bogo', 'informational', 'discount']

Several things to notice from above:
* There are 10 different offers in `portfolio`;
* The values of the 'reward' columns span from 0 to 10, and it is an integer, with the mean reward of 4.20;
* Difficulty (minimum required spend to complete an offer) ranges from 0 to 20 (also an int), with a mean difficulty being 7.70;
* Duration of an offer, i.e. number of days that an offer will be active, is up to 10 days, with an average duration of 6.50 days;
* There are three different offer types: bogo, information, and discount;
* Informational offers have both difficulty and reward of 0;
* There are no incomplete rows in the `portfolio` table.

#### Profile

In [8]:
profile.shape

(17000, 5)

In [9]:
profile.head(3)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN


In [10]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


From the preliminary analysis of the demographic data from the table `profile`, we can observe:
* The table `profile` contains 17,000 rows, i.e. 17,000 different customer profiles - could any customers be the same?;
* There are some incomplete data rows;
* Mean age of a Starbucks customer is 62.53, with maximum of 118 years. Possibility of erroneous data (e.g. customer giving fake age when subscribing)?;
* Column `became_member_on` is in format `YYYYMMDD`, as integer;
* Mean income of starbucks customers is 65,404.99 (I presume $).

#### Transcript

In [11]:
transcript.shape

(306534, 4)

In [12]:
transcript.head(3)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0


From the preliminary analysis of the demographic data from the table `transcript`, we can observe:

* There are 306,534 recorced transactions, each described with 4 features: person (customer id as in `profile`), event (the type of transaction that is recorded), value (a dict), and time (test begins at t=0).

## Exploratory Data Analysis (EDA)

More detailed EDA will be performed using pandas-profiling package: https://github.com/pandas-profiling/pandas-profiling

In [13]:
pandas_profile_portfolio = ProfileReport(portfolio, title="Pandas Profiling Report for table 'portfolio' ")

In [14]:
pandas_profile_portfolio

In [15]:
#pandas_profile_portfolio.to_file("pandas_profile_table_portfolio.html")

In [16]:
channels_list = list(np.unique([y for x in portfolio['channels'] for y in x]))
channels_list

['email', 'mobile', 'social', 'web']

Remarks:
* There are no duplicate rows or missing values.
* 6 fields in dataframe: 3 numerical (reward, difficulty, duration), 2 categorical (offer_type, id), and 'channels' which is a list of strings (type not recognised by pandas-profiling).
* Difficulty and reward have similar histograms. 
* There are 4 possible channels via which an offer can be sent: ['email', 'mobile', 'social', 'web']
* There is a positive correlation between all numerical variables; however, while duration and reward are weakly correlated, reward and difficulty are fairly correlated, and duration and difficulty are quite strongly correlated.

In [17]:
pandas_profile_profile = ProfileReport(profile, title="Pandas Profiling Report for table 'profile' ")

In [18]:
#pandas_profile_profile.to_file("pandas_profile_table_profile.html")

In [19]:
pandas_profile_profile

In [20]:
# how many rows have missing valus
n_missing_profiles = profile.shape[0] - profile.dropna().shape[0]
print("There are ", n_missing_profiles, " rows with at least one missing value.")

There are  2175  rows with at least one missing value.


In [21]:
profiles_over_100 = profile[profile['age'] > 100]
print("There are ", profiles_over_100.shape[0], " profiles with age over 100.")
print("Among the profiles with age over 100, there are ", 
      profiles_over_100.shape[0] - profiles_over_100.dropna().shape[0], " profiles with missing values.")

profiles_over_100.dropna()

There are  2180  profiles with age over 100.
Among the profiles with age over 100, there are  2175  profiles with missing values.


,gender,age,id,became_member_on,income
1556,F,101,4d2ccfcbbebf4bd9baf4b7e433d0e288,20171004,43000.0
4100,F,101,d2fdc2be8ab64e4ba04830d441e53fd5,20150526,99000.0
14846,F,101,e0ea90ddd2f147e082d21e97f51ec1b1,20171109,56000.0
15800,F,101,047ad0135cfe4c0ea5ba019da4de9c52,20170309,59000.0
16864,F,101,1593d617fac246ef8e50dbb0ffd77f5f,20171127,82000.0


Observations:
* There are 5 variables: 3 numerical (age, became_member_on, income), and 2 categorical (gender, id)
* 'Id' column has unique values, as it should be
* There no duplicate rows and 4350 missing values (around 5% of all cells in the table). The missing values occur in columns 'gender' and 'income', both with 2175 missing values. Further inspections showed that they all occur in same 2175 rows, that is, 'gender' and 'income' are always missing simultaneously.
* 'Gender' has three categories: M (8484 values), F (6129 values), and O (212 values).
* There are 85 distinct ages, with a surprisingly large number of customer with age over 100 (2180 profiles to be precise). In fact, inspecting further the dataframe `profiles` discovered that all the rows with missing values are in fact aged over 100, and as such the best would be to discard them from further analysis as outliers/anomalies. That leaves us 5 female profiles aged over 100 (concretely, age 101) with varying incomes.
* Age and income, as expcted, have a weaker positive correlation, while became_member_on is weakly negatively correlated with income and age. This means that there is a slight tendency that customers that joined more recently the Starbucks programme will tend to be slightly younger and naturally of lower income, which is again as one would expect.

In [22]:
pandas_profile_transcript = ProfileReport(transcript, title="Pandas Profiling Report for table 'transcript' ")

In [23]:
#pandas_profile_transcript.to_file("pandas_profile_table_transcript.html")

In [24]:
pandas_profile_transcript

In [25]:
keys_transactions = np.unique([y for x in list(transcript[transcript.event == 'transaction']['value']) for y in x.keys()])
list(keys_transactions)

['amount']

In [26]:
keys_non_transactions = np.unique([y for x in list(transcript[transcript.event != 'transaction']['value']) for y in x.keys()])
list(keys_non_transactions)

['offer id', 'offer_id', 'reward']

In [27]:
list(transcript[transcript.event == 'offer completed']['value'][:5])

[{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2},
 {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2},
 {'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9', 'reward': 5},
 {'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd', 'reward': 10},
 {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0', 'reward': 10}]

Observations:
* There are no missing values in the table `transcript`.
* There are 2962 duplicate rows, which is not very surprising since one client can perform some transactions multiple times.
* The variable 'event' is categorical, with for possible categories: ** [transaction, offer received, offer viewed, offer completed]**.
* Most common event type is *transaction*, recorded 138,953 times (around 45% of all the events), followed by offer received (76,277 records or 25%), offer viewed (around 19%), and lastly offer completed (around 11%). This means that around 76% of the offer received get viewed by a customer and around 44% of the received offers are completed.
* In the 'person' variable, there aer 17,000 different categories - which corresponds to the number of different customer profiles registered, meaning that every customer has at least one event recorded. In fact, 5.5% of the customer profiles appear only once in this table (935 profiles), i.e. they have only one event assigned to them. A customer profile with most occurrences in the table has 51 events assigned to them.
* If the event type is 'transaction', then the only key in the dictionary of the variable 'value' is amount (of that transaction). Otherwise, the possible keys are:
    * **offer id / offer_id - NB: this needs to be uniformised as it is the same key in fact (id of an offer)!!**
    * **reward** - the reward a customer received in case of completing an offer
* Time, which is the number of hours since the test started, has 120 distinct values, spaning from 0 to 714 hours (29.75 days).


## Data cleaning

The tables I am working with are pretty clean already. During the exploratory analysis, several things were identified as necessary to perform to make datasets suitable for further analysis and feature creation:
* drop any incomplete rows of data in the table `profile`
* uniformise the keys 'offer_id' and 'offer id' in the table `transcript`, column event_type --> they are the same key
* the table `portfolio` needs no cleaning

In [34]:
print('(orig profile) rows, cols: ', profile.shape)

# drop any incomplete data, set client id as index column
clean_profile = profile.dropna(axis=0).set_index("id")
print('(clean profile) rows, cols: ', clean_profile.shape)

(orig profile) rows, cols:  (17000, 5)
(clean profile) rows, cols:  (14825, 4)


In [35]:
#for x in list(transcript[transcript.event != 'transaction']['value']) for y in x.keys()

# uniform keys in transcript/event: offer_id / offer id
"""
TO DO!!!

"""

'\nTO DO!!!\n\n'

In [38]:
# became_member_on to datetime format
clean_profile["became_member_on"] = pd.to_datetime(clean_profile["became_member_on"], format='%Y%m%d')

In [118]:
clean_profile.head()

,gender,age,became_member_on,income,membership_duration,income_age,num_transactions,total_transactions,avg_per_transaction,age_group,buying_frequency
id,,,,,,,,,,,
0610b486422d4921ae7d2bf64640c50b,F,55,2017-07-15,112000.0,377,2036.363636,6,77.01,12.835000,4.0,62.833333
78afa995795e4d85b5d9ceeca43f5fef,F,75,2017-05-09,100000.0,444,1333.333333,18,159.27,8.848333,5.0,24.666667
e2127556f4f64592b11af22de27a7932,M,68,2018-04-26,70000.0,92,1029.411765,12,57.73,4.810833,5.0,7.666667
389bc3fa690240e798340f5a15918d5c,M,65,2018-02-09,53000.0,168,815.384615,20,36.43,1.821500,5.0,8.400000
2eeac8d8feae4a8cad5a6af0499a211d,M,58,2017-11-11,51000.0,258,879.310345,10,15.62,1.562000,4.0,25.800000


In [119]:
print("The newest memeber in the database became a member on ", max(clean_profile.became_member_on))
print("The latest transaction in the database has been performed", max(transcript.time), \
      "days since the beginning of test.")

The newest memeber in the database became a member on  2018-07-26 00:00:00
The latest transaction in the database has been performed 714 days since the beginning of test.


In [120]:
def calculate_profile_metrics(df_profile):
    """
    It calculates a series of metrics from the cleaned profile dataframe.
    
    Returns a new dataframe with the original and newly calculated columns.
    """
    # calculate number of days someone has been a member starting from the reference day

    ref_date = max(df_profile.became_member_on) + pd.DateOffset(1) #set reference day as the latest day + 1
    print("The date set as a reference is ", ref_date)

    # calculate membership duration w.r.t. ref_date
    df_profile['membership_duration'] = (ref_date - df_profile['became_member_on']).dt.days

    # metric of income relative to their age --> MOVE THIS TO FEATURE ENGINEERING!
    df_profile["income_age"] = df_profile['income'] / df_profile['age']


    """
    categorise by common age groups, as follows:
    18-24: group 1 (young adults, commonly uni),
    25-34: group 2 (young professionals),
    35-54: group 3 (middle age),
    55-64: group 4 (older adults),
    65+: group 5 (retirement).in
    """

    age_bins = [18, 24, 34, 54, 64, 118]
    df_profile['age_group'] = pd.cut(df_profile['age'], age_bins, right=True, labels = False) + 1

    return df_profile

In [121]:
def split_transcripts(df_transcript):
    """
    This function splits the transcripts dataframe into transactions and offers, and
    returns the two corresponding dataframes.
    """
    
    # extract just transcripts that were transactions
    df_transactions = df_transcript[df_transcript['event'] == 'transaction']
    
    # extract the amount of each transactions from the dict in 'value'
    df_transactions['amount'] = df_transactions['value'].apply(lambda x: x['amount'])
    
    # offer transcripts
    df_offers = df_transcript[df_transcript['event'] == 'transaction']
    
    return df_transactions, df_offers

In [123]:
def calculate_profile_transaction_metrics(df_profile, df_transactions, df_transcripts):
    """
    It calculates a series of metrics using the cleaned profile dataframe, and the dataframe on
    customer transactions.
    
    Returns a new dataframe with the original and newly calculated columns.
    """
    
    # count number of all transcripts (offer e) of each customer
    df_profile['num_trancripts'] = df_transcripts['person'].value_counts()
    df_profile['num_trancripts_norm'] = df_transcripts['person'].value_counts(normalize=True) # normalised

    # count just the transactions a member did since the beginning of the test
    df_profile['num_transactions'] = df_transactions['person'].value_counts()

    # sum all the transactions a customer made during the test
    df_profile['total_transactions'] = df_transactions.groupby('person')['amount'].sum()

    # add average per transaction
    df_profile['avg_per_transaction'] = df_profile['total_transactions'] / df_profile['num_transactions']

    # buying frequency: average days between transactions
    df_profile['buying_frequency'] = df_profile['membership_duration'] / df_profile['num_transactions']
    
    return df_profile


In [132]:
df_transactions, df_offers = split_transcripts(transcript)
profile_temp = calculate_profile_metrics(clean_profile)
profile_final = calculate_profile_transaction_metrics(profile_temp, df_transactions, transcript)

The date set as a reference is  2018-07-27 00:00:00


In [133]:
profile_final.head()

,gender,age,became_member_on,income,membership_duration,income_age,num_transactions,total_transactions,avg_per_transaction,age_group,buying_frequency,num_trancripts,num_trancripts_norm
id,,,,,,,,,,,,,
0610b486422d4921ae7d2bf64640c50b,F,55,2017-07-15,112000.0,377,2036.363636,3.0,77.01,25.670000,4.0,125.666667,6,0.000020
78afa995795e4d85b5d9ceeca43f5fef,F,75,2017-05-09,100000.0,444,1333.333333,7.0,159.27,22.752857,5.0,63.428571,18,0.000059
e2127556f4f64592b11af22de27a7932,M,68,2018-04-26,70000.0,92,1029.411765,3.0,57.73,19.243333,5.0,30.666667,12,0.000039
389bc3fa690240e798340f5a15918d5c,M,65,2018-02-09,53000.0,168,815.384615,3.0,36.43,12.143333,5.0,56.000000,20,0.000065
2eeac8d8feae4a8cad5a6af0499a211d,M,58,2017-11-11,51000.0,258,879.310345,4.0,15.62,3.905000,4.0,64.500000,10,0.000033


## Visualizing the data

In [134]:
"""
TO DO!
"""

'\nTO DO!\n'

## Prepare the features (profile_final) for clustering

In [135]:
# map gender to 0/1: 0 - female, 1 - male, 2 - O
gender_mapping_int = {"gender": {'F':0, 'M':1, 'O':2}}
profile_final = profile_final.replace(gender_mapping_int)

In [136]:
# drop the features that are not going to be used in clustering
profile_final.drop('became_member_on', axis=1, inplace=True)

In [138]:
profile_final.head()

,gender,age,income,membership_duration,income_age,num_transactions,total_transactions,avg_per_transaction,age_group,buying_frequency,num_trancripts,num_trancripts_norm
id,,,,,,,,,,,,
0610b486422d4921ae7d2bf64640c50b,0,55,112000.0,377,2036.363636,3.0,77.01,25.670000,4.0,125.666667,6,0.000020
78afa995795e4d85b5d9ceeca43f5fef,0,75,100000.0,444,1333.333333,7.0,159.27,22.752857,5.0,63.428571,18,0.000059
e2127556f4f64592b11af22de27a7932,1,68,70000.0,92,1029.411765,3.0,57.73,19.243333,5.0,30.666667,12,0.000039
389bc3fa690240e798340f5a15918d5c,1,65,53000.0,168,815.384615,3.0,36.43,12.143333,5.0,56.000000,20,0.000065
2eeac8d8feae4a8cad5a6af0499a211d,1,58,51000.0,258,879.310345,4.0,15.62,3.905000,4.0,64.500000,10,0.000033


In [141]:
# normalize the data into a normalized range, 0-1

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

profile_scaled = pd.DataFrame(scaler.fit_transform(profile_final.astype(float)))


profile_scaled.columns = profile_final.columns
profile_scaled.index = profile_final.index

In [142]:
profile_scaled.head()

,gender,age,income,membership_duration,income_age,num_transactions,total_transactions,avg_per_transaction,age_group,buying_frequency,num_trancripts,num_trancripts_norm
id,,,,,,,,,,,,
0610b486422d4921ae7d2bf64640c50b,0.0,0.445783,0.911111,0.206253,0.459526,0.057143,0.047782,0.056545,0.75,0.074979,0.081633,0.081633
78afa995795e4d85b5d9ceeca43f5fef,0.0,0.686747,0.777778,0.243006,0.271348,0.171429,0.098922,0.050082,1.00,0.037820,0.326531,0.326531
e2127556f4f64592b11af22de27a7932,0.5,0.602410,0.444444,0.049918,0.189997,0.057143,0.035796,0.042306,1.00,0.018260,0.204082,0.204082
389bc3fa690240e798340f5a15918d5c,0.5,0.566265,0.255556,0.091607,0.132709,0.057143,0.022555,0.026574,1.00,0.033385,0.367347,0.367347
2eeac8d8feae4a8cad5a6af0499a211d,0.5,0.481928,0.233333,0.140976,0.149820,0.085714,0.009617,0.008320,0.75,0.038460,0.163265,0.163265


In [143]:
profile_scaled.describe()

,gender,age,income,membership_duration,income_age,num_transactions,total_transactions,avg_per_transaction,age_group,buying_frequency,num_trancripts,num_trancripts_norm
count,14825.000000,14825.000000,14825.000000,14825.000000,14825.000000,14492.000000,14492.000000,14492.000000,14755.00000,14492.000000,14825.000000,14825.000000
mean,0.300438,0.438476,0.393389,0.286604,0.265914,0.215814,0.074333,0.033462,0.65183,0.044756,0.334669,0.334669
std,0.259057,0.209442,0.239981,0.229953,0.146119,0.145220,0.080955,0.036656,0.28849,0.046445,0.141338,0.141338
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.000000,0.289157,0.211111,0.114098,0.160422,0.114286,0.021160,0.008289,0.50000,0.017947,0.224490,0.224490
50%,0.500000,0.445783,0.377778,0.196380,0.245855,0.171429,0.055771,0.031377,0.75000,0.033086,0.326531,0.326531
75%,0.500000,0.578313,0.555556,0.437191,0.343971,0.285714,0.101085,0.047234,1.00000,0.056434,0.428571,0.428571
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [149]:
#plt.matshow(profile_scaled.corr())
profile_scaled.corr()

,gender,age,income,membership_duration,income_age,num_transactions,total_transactions,avg_per_transaction,age_group,buying_frequency,num_trancripts,num_trancripts_norm
gender,1.000000,-0.144653,-0.220746,0.015424,-0.026465,0.050220,-0.147693,-0.176760,-0.125983,-0.044677,-0.004970,-0.004970
age,-0.144653,1.000000,0.306703,0.012300,-0.633946,-0.153688,0.112301,0.200940,0.942218,0.123109,-0.084491,-0.084491
income,-0.220746,0.306703,1.000000,0.025769,0.438747,-0.262944,0.329583,0.492556,0.288868,0.218226,-0.131375,-0.131375
membership_duration,0.015424,0.012300,0.025769,1.000000,0.007375,0.415941,0.154500,0.021810,0.012200,0.521127,0.372743,0.372743
income_age,-0.026465,-0.633946,0.438747,0.007375,1.000000,-0.032719,0.126657,0.147270,-0.623999,0.029770,-0.007801,-0.007801
num_transactions,0.050220,-0.153688,-0.262944,0.415941,-0.032719,1.000000,0.326972,-0.127771,-0.143157,-0.282403,0.902984,0.902984
total_transactions,-0.147693,0.112301,0.329583,0.154500,0.126657,0.326972,1.000000,0.756973,0.111494,-0.073562,0.418926,0.418926
avg_per_transaction,-0.176760,0.200940,0.492556,0.021810,0.147270,-0.127771,0.756973,1.000000,0.193705,0.132327,-0.005575,-0.005575
age_group,-0.125983,0.942218,0.288868,0.012200,-0.623999,-0.143157,0.111494,0.193705,1.000000,0.115994,-0.076579,-0.076579
buying_frequency,-0.044677,0.123109,0.218226,0.521127,0.029770,-0.282403,-0.073562,0.132327,0.115994,1.000000,-0.263955,-0.263955


# Data Modelling

## Dimensionality reduction

IMPLEMENT IF SEEN AS NEEDED!!

# Market segmentation

In this part, I will use k-mean to segment the customers using the above extracted features that describe their basic purschasing patters (excluding the data on how they react to offers). To choose a "good k", I use the elbow method.
Due to a fairly small feature space, the clustering is performed directly on the created features without using dimensionality reduction (PCA).

In [150]:
"""from sagemaker import get_execution_role

session = sagemaker.Session() # store the current SageMaker session

# get IAM role
role = get_execution_role()
print(role)
"""

'from sagemaker import get_execution_role\n\nsession = sagemaker.Session() # store the current SageMaker session\n\n# get IAM role\nrole = get_execution_role()\nprint(role)\n'

In [151]:
"""
# get default bucket
bucket_name = session.default_bucket()
print(bucket_name)
print()
"""

'\n# get default bucket\nbucket_name = session.default_bucket()\nprint(bucket_name)\nprint()\n'

In [152]:
# define location to store model artifacts
prefix = 'starbucks'

output_path='s3://{}/{}/'.format(bucket_name, prefix)

print('Training artifacts will be uploaded to: {}'.format(output_path))

NameError: name 'bucket_name' is not defined

In [155]:
# define a KMeans estimator
from sagemaker import KMeans

ModuleNotFoundError: No module named 'sagemaker'

The dataframe with features needs to be transformd into a numpr array and then into a RecordSet, for the KMeans model.

In [154]:
# convert the transformed dataframe into record_set data
kmeans_train_data_np = profile_scaled.values.astype('float32')
kmeans_formatted_data = kmeans.record_set(kmeans_train_data_np)

NameError: name 'kmeans' is not defined

### Train the k-means model

The training of the k-means model is done on a range of K's. That is so that I am able to visualise the elbow graph and select a good k.

In [157]:
K = range(2, 10)  # start from 3/5 and up to 10/12 to see the elbow graph
training_jobs = [] # list of the traning jobs, for extracting model artifacts

In [ ]:
for k in K:
    print('starting train job:' + str(k))
    
    kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.c4.xlarge',
                output_path=output_path, # using the same output path as was defined, earlier              
                k=k)
    %%time
    # train kmeans
    kmeans.fit(kmeans_formatted_data)
    training_jobs.append(my_estimator.latest_training_job.name)

In [ ]:
plt.plot()
colors = ['b', 'g', 'r']
markers = ['o', 'v', 's']
models = {}
distortions = []

for k in K:
    #s3_client = boto3.client('s3')
    kmeans_job_name = training_jobs[k - K[0]]
    model_key = os.path.join(prefix, kmeans_job_name, 'output/model.tar.gz')
    
    # download the model file
    boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')
    os.system('tar -zxvf model.tar.gz')
    os.system('unzip model_algo-1')
    
    
    # get the trained kmeans params using mxnet
    kmeans_model_params = mx.ndarray.load('model_algo-1')

    kmeans_numpy = kmeans_model_paramas[0].asnumpy()
    distortions.append(sum(np.min(cdist(train_data, kmeans_numpy, 'euclidean'), axis=1)) / train_data.shape[0])
    models[k] = kmeans_numpy
 
# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('distortion')
plt.title('Elbow graph')
plt.show()